Scraper for the first page of booking.com

lat_lon coresponds to latitude and longitude

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service

In [2]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://www.booking.com/searchresults.en-gb.html?label=gen173nr-1BCAEoggI46AdIM1gEaLYBiAEBmAEJuAEHyAEN2AEB6AEBiAIBqAIDuALt1qChBsACAdICJDA2NjU4ZDE2LTQyMDMtNGIwYS1hYzE3LTE3MGI0NTM3ZmE5YtgCBeACAQ&sid=de8af4c9f85c3767c525594abc958387&aid=304142&ss=Lisbon%2C+Lisbon+Region%2C+Portugal&ssne=Lisbon&ssne_untouched=Lisbon&lang=en-gb&src=index&dest_id=-2167973&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=1da15bb6e9c40401&ac_meta=GhAxZGExNWJiNmU5YzQwNDAxIAAoATICZW46Bmxpc2JvbkAASgBQAA%3D%3D&checkin=2023-06-19&checkout=2023-06-24&group_adults=1&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bht_id%3D203')

In [3]:
#Scroll down
screen_height = driver.execute_script("return window.screen.height;") 
i = 1
while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(0.5)
    scroll_height = driver.execute_script("return document.body.scrollHeight;") 
    if (screen_height) * i > scroll_height:
        break

In [4]:
titles = driver.find_elements(By.CSS_SELECTOR, 'div.f6431b446c.a15b38c233')
titles = list(map(lambda i: i.text, titles))


### Some offers don't have any reviews, so it is necessary to check
containers = driver.find_elements(By.CSS_SELECTOR, 'div.c624d7469d.a0e60936ad.a3214e5942.b0db0e8ada') ### Headlines of all offers
containers = list(map(lambda i: i.text, containers))

quantity_opinions = driver.find_elements(By.CSS_SELECTOR, 'div.abf093bdfe.f45d8e4c32.d935416c47') ### Couunts of all review, if they occur
quantity_opinions = list(map(lambda i: i.text, quantity_opinions))

opinions_final = []

x = 0
for i in range(0, len(titles)):
    if ' reviews' in containers[i]: ### If offer have reviews
        opinions_final.append(quantity_opinions[x])
        x += 1
    else: ### If offer doesn't have reviews
        opinions_final.append(0)



links = driver.find_elements(By.CSS_SELECTOR, 'a.a78ca197d0')
links = list(map(lambda i: i.get_attribute('href'), links))

In [5]:
lat_lon = []
driver.find_element(By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler').click() #Accepting cookies

for link in links:
    driver.get(link)
    maps=driver.find_element(By.CSS_SELECTOR, 'button.txp-map-cta.bui-button.bui-button--primary').click()
    latlon=driver.find_element(By.ID, 'hotel_address').get_attribute('data-atlas-latlng')
    lat_lon.append(latlon)

In [6]:
driver.quit()

In [7]:
dict = {'Titles':titles, 'Number of opinions':opinions_final,'Link':links, 'Latitude, longitude':lat_lon}
df = pd.DataFrame(dict)
display(df)

Titles Number of opinions  \
0                        Avenue Hostel & Suites      3,306 reviews   
1                               Ambiente Hostel      1,162 reviews   
2                             Lisbon Top Hostel      1,440 reviews   
3                            Green Heart Hostel      3,504 reviews   
4        Alojamento Local Private Accommodation        822 reviews   
..                                          ...                ...   
70                     Hotel Travel Park Lisboa      3,311 reviews   
71                           We Hate F Tourists      2,638 reviews   
72  HI Parque das Nações – Pousada de Juventude        650 reviews   
73                         Jupiter Lisboa Hotel      4,958 reviews   
74                                   The Bridge      1,267 reviews   

                                                 Link      Latitude, longitude  
0   https://www.booking.com/hotel/pt/avenue-rooms-...  38.72401692,-9.14753024  
1   https://www.booking.com/hotel/pt/ambiente-host...  38.72927591,-9.13449915  
2   https://www.booking.com/hotel/pt/lisbon-top-ho...  38.72929798,-9.14945634  
3   https://www.booking.com/hotel/pt/liv-39-in-lis...  38.72335600,-9.14578130  
4   https://www.booking.com/hotel/pt/private-accom...  38.72351770,-9.13314700  
..                                                ...                      ...  
70  https://www.booking.com/hotel/pt/travelparklis...  38.72713613,-9.13469598  
71  https://www.booking.com/hotel/pt/we-hate-f-tou...  38.72223100,-9.13660000  
72  https://www.booking.com/hotel/pt/pousada-de-ju...  38.77811728,-9.09822188  
73  https://www.booking.com/hotel/pt/jupiter-lisbo...  38.73953362,-9.14586602  
74  https://www.booking.com/hotel/pt/the-bridge-li...  38.69975200,-9.19220300  

[75 rows x 4 columns]